# Conflict-Free Replicated Datatypes

In this tutorial, you will be introduced to Antidote Databases. You can test and see how an eventually consistent database works. We will guide you through the Antidote CRDT data types, transactions, and how Antidote resolves inconsistencies. For more information, see http://syncfree.github.io/antidote/.

> Note that this Jupyter notebook is based on Antidote Java API, but it is tailored for tutorial purpose. As a result, this tutorial might not reflect the full capabilities of Antidote. To find out how to use the Java API, please refer to the __[official Antidote Java API](https://www.javadoc.io/doc/eu.antidotedb/antidote-java-client/0.1.0)__.

First of all, you need to make sure that Antidote service is up and running.

In [ ]:
init()

# Data types in Antidote

Antidote supports several Conflict-free replicated data types (CRDT). Each type handles inconsistencies differently. In this section, we show some examples of how you can use these data types. The data types supported by Antidote include:
- __Number types__
    - Counter
- __Registers__
    - Last-writer wins register
    - Multi-value register
- __Sets__
    - Add-wins set (or Observed-remove set)
    - Remove-win set
- __Maps__
    - Grow-only map
    - Remove-resets map

You need a key in order to update/remove/reset/read your data object in an Antidote bucket. A bucket is an internal data structure used by Antidote where objects are stored. In this tutorial, we have one bucket which uses a unique session id as its key. An object key should be unique within the bucket, regardless of data types.

### Counters

The simplest CRDT is probably a Counter. First, we need to create a key for our counter object. We use `"c1"` as a unique key for accessing this Counter object. `getCounterKey("c1")` returns a Counter Key object with key `"c1"`. We assign this Counter Key object to `counter_key` for an easy reference to our `"c1"` key later on.

In [2]:
counter_key = getCounterKey("c1")

COUNTER_c1

Counter is simply an integer object with an increment and decrement operation. The initial value of a counter is 0. Let's try incrementing our `c1` counter by 10, and decrementing it by 1.

In [3]:
applyUpdate(incrementCounter(counter_key, 10))
applyUpdate(incrementCounter(counter_key, -1))

08:40:42 Updated key 'COUNTER_c1' on Antidote 1

Reading a Counter object returns the aggregated value from all received operations. In our example, the `c1` counter should now be equal to 9.

In [4]:
read(counter_key)

9

### Registers

Register can store an arbitrary single-valued object. We use a register object to store a String in the following example.

In [5]:
register_key = getLWWRegisterKey("r1")

LWWREG_r1

In [6]:
applyUpdate(assignLWWRegister(register_key, "Hello"))

08:40:45 Updated key 'LWWREG_r1' on Antidote 1

In [7]:
read(register_key)

Hello

### Sets

Set can store a collection of values of the same type. This notebook supports only a Set of Strings.

In [8]:
set_key = getSetKey("s1")

ORSET_s1

In [9]:
applyUpdate(addToSet(set_key, "A","B","C","D"))

08:40:49 Updated key 'ORSET_s1' on Antidote 1

In [10]:
read(set_key)

[A, B, C, D]

In [11]:
applyUpdate(removeFromSet(set_key, "C","D"))

08:40:52 Updated key 'ORSET_s1' on Antidote 1

In [12]:
read(set_key)

[A, B]

### Maps

Map associates a collection of identifiers with different objects. In the following example, we create a map which stores two element objects: a counter, and a set object.

In [15]:
map_key = getRRMapKey("m1")
read(map_key)

java.io.IOException: End of input while data expected

In [14]:
counter_key = getCounterKey("c2")

COUNTER_c2

In [15]:
set_key = getSetKey("s2")

ORSET_s2

To update a map, we actually update element objects of the map.

In [18]:
applyUpdate(updateRRMap(map_key, incrementCounter(counter_key, 1) , addToSet(set_key, "X", "Y")))

java.net.SocketException: Broken pipe (Write failed)

To read from a map, we need to specify the identifier of the element object.

In [18]:
readFromMap(map_key, counter_key)

java.net.SocketException: Broken pipe (Write failed)

In [ ]:
readFromMap(map_key, set_key)

In [ ]:
applyUpdate(removeFromRRMap(map_key, counter_key))

In [0]:
readFromMap(map_key, counter_key)

No such property: map_key for class: Script1

In [0]:
readFromMap(map_key, set_key)

No such property: map_key for class: Script2

We can also recursively nest maps in a map:

In [ ]:
child_map_key = getRRMapKey("m1.1")

In [ ]:
nested_counter_key = getCounterKey("c2.1")

In [ ]:
applyUpdate(updateRRMap(map_key, updateRRMap(child_map_key, 
                            incrementCounter(child_counter_key, 43))))

Reading a map in a map will return a mapResult object

In [ ]:
mapResult = readFromMap(map_key, child_map_key)

mapResults can be read separately. And it is possible to use it recursively to read map results. 

In [ ]:
readFromMapResult(mapResult, child_counter_key)